<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Medical_MULTIPLE_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai -q
!pip install anthropic -q
!pip install colab-env -q

In [3]:
import os
from typing import Dict, Any, List, Optional
import google.generativeai as genai
from anthropic import Anthropic
import colab_env # For Google Colab

# Import necessary libraries
try:
    import google.generativeai as genai
    from anthropic import Anthropic
    from anthropic.types import MessageParam
    # Conditional import for Colab's userdata
    try:
        from google.colab import userdata
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
        print("Running outside Google Colab. 'userdata.get' will not be available.")

except ImportError:
    print("\n[CRITICAL ERROR] Please install necessary libraries:")
    print("pip install google-generativeai anthropic")
    print("Exiting as API functionality cannot be demonstrated without them.")
    # In a real application, you might raise an exception or handle this more gracefully
    # For this example, we'll allow execution but API calls will be mocked
    pass


# --- 1. Core Components (Interfaces and Implementations with API Management) ---

class BaseModel:
    """Base class for Language Models and Reasoning Models."""
    def __init__(self, name: str, identifier: str, api_client: Any):
        self.name = name
        self.identifier = identifier # The specific model string
        self.api_client = api_client # The actual API client instance

    def generate(self, prompt: str, **kwargs) -> str:
        raise NotImplementedError("Subclasses must implement 'generate'")

class LargeLanguageModel(BaseModel):
    """Integrates with a general LLM API (e.g., Gemini)."""
    def __init__(self, name: str, identifier: str, api_client: Optional[genai.GenerativeModel]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, genai.GenerativeModel):
             print(f"Warning: Expected genai.GenerativeModel for {identifier}, but received {type(api_client)}. Mocking API calls.")
             self.api_client = None

    def generate(self, prompt: str, temperature: float = 0.7, max_tokens: int = 500) -> str:
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to generate response for prompt: '{prompt[:80]}...'")

        if self.api_client:
            try:
                generation_config = {
                    "temperature": temperature,
                    "max_output_tokens": max_tokens,
                }
                response = self.api_client.generate_content(
                    prompt,
                    generation_config=generation_config
                )
                if response and hasattr(response, 'text') and response.text:
                    return response.text
                elif response and hasattr(response, 'prompt_feedback'):
                    print(f"Prompt feedback from LLM: {response.prompt_feedback}")
                    return f"API did not generate content due to safety concerns or other issues."
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling LLM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Generating response for prompt: '{prompt[:80]}...'")
        if "patient symptoms" in prompt.lower() and "lab results" in prompt.lower():
            return f"[{self.name}] (MOCK) Analyzing patient symptoms and lab results. Key findings: Elevated white blood cell count, fever. Need to retrieve medical history and differential diagnoses."
        elif "treatment options" in prompt.lower() and "condition x" in prompt.lower():
             return f"[{self.name}] (MOCK) Considering treatment options for Condition X based on patient profile. Potential treatments include medication A, therapy B, or surgery C. Need to retrieve clinical guidelines."
        return f"[{self.name}] (MOCK) Generated a general medical response based on: '{prompt}'"


class LargeReasoningModel(BaseModel):
    """Integrates with a specialized medical reasoning API (e.g., Claude Sonnet)."""
    def __init__(self, name: str, identifier: str, api_client: Optional[Anthropic]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, Anthropic):
             print(f"Warning: Expected anthropic.Anthropic for {identifier}, but received {type(api_client)}. Mocking API calls.")
             self.api_client = None


    def generate(self, prompt: str, complexity_level: str = "high") -> str:
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to perform medical reasoning for prompt: '{prompt[:80]}...' (Complexity: {complexity_level})")

        if self.api_client:
            try:
                messages: List[MessageParam] = [
                    {"role": "user", "content": prompt}
                ]
                response = self.api_client.messages.create(
                    model=self.identifier,
                    max_tokens=1000,
                    messages=messages,
                    temperature=0.1, # Low temperature for deterministic reasoning
                )
                if response and hasattr(response, 'content') and response.content:
                    return "".join(block.text for block in response.content if hasattr(block, 'text'))
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling LRM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Performing medical reasoning for prompt: '{prompt[:80]}...'")
        if "analyze patient data" in prompt.lower() and "diagnosis" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Analyzed patient data. Differential diagnoses include bacterial infection (supported by high WBC) and viral infection. Further tests recommended: Blood culture, imaging."
        elif "recommend treatment" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Evaluated patient's condition and retrieved guidelines. Recommended starting broad-spectrum antibiotics if bacterial infection is suspected, pending culture results. Advise supportive care."
        return f"[{self.name}] (MOCK) LRM performed complex medical reasoning based on: '{prompt}'"

# --- Mock Retrieval Tools (Adapted for Medicine) ---

class Memory:
    """Manages short-term and long-term memory."""
    def __init__(self):
        self.short_term_memory: List[str] = []
        self.long_term_memory: List[str] = [] # Could be a more complex data structure

    def add_short_term(self, entry: str):
        self.short_term_memory.append(entry)
        # Keep short-term memory limited
        if len(self.short_term_memory) > 10:
            self.short_term_memory.pop(0)

    def add_long_term(self, entry: str):
        self.long_term_memory.append(entry)
        # In a real system, this might involve embeddings and indexing for medical data

    def retrieve_short_term(self, query: str = "") -> List[str]:
        # Simple retrieval; in real system, would be more sophisticated (e.g., semantic search)
        return [m for m in self.short_term_memory if query.lower() in m.lower()]

    def retrieve_long_term(self, query: str = "") -> List[str]:
        return [m for m in self.long_term_memory if query.lower() in m.lower()]

class VectorDatabase:
    """Mock Vector Database (Adapted for Medicine)."""
    def query(self, embedding: List[float], top_k: int = 5) -> List[str]:
        print(f"Querying Vector DB with embedding (simulating medical data search)...")
        # Simulate results related to medical data
        return ["patient_record_xyz_vdb", "clinical_trial_data_abc_vdb"]

class SemanticDatabase:
    """Mock Semantic Database (Adapted for Medicine)."""
    def query(self, natural_language_query: str) -> List[str]:
        print(f"Querying Semantic DB for: '{natural_language_query}' (simulating medical knowledge search)...")
        # Simulate results related to medical guidelines or concepts
        if "guidelines" in natural_language_query or "protocol" in natural_language_query:
            return ["Treatment guidelines for pneumonia (IDSA).", "Protocol for managing sepsis."]
        elif "drug interactions" in natural_language_query or "side effects" in natural_language_query:
             return ["Known interactions for drug X.", "Common side effects of medication Y."]
        return ["medical_concept_1_sdb", "disease_definition_2_sdb"]

class SearchEngine:
    """Mock Search Engine (e.g., PubMed, Medical News Sites - Adapted)."""
    def search(self, query: str) -> List[str]:
        print(f"Searching external sources for: '{query}' (simulating medical web search)...")
        if "latest research covid treatment" in query.lower():
            return ["Recent study on antiviral efficacy for COVID.", "News: FDA approves new treatment for COVID-19."]
        elif "symptoms of condition z" in query.lower():
            return ["Common symptoms of Condition Z.", "Differential diagnoses for symptoms similar to Condition Z."]
        return [f"Medical web result for '{query}' - Article A", f"Medical web result for '{query}' - Study B"]

# --- 2. Retrieval Agents ---

class RetrievalAgent:
    """Base class for retrieval agents."""
    def __init__(self, name: str):
        self.name = name

    def retrieve(self, query: str, **kwargs) -> List[str]:
        raise NotImplementedError("Subclasses must implement 'retrieve'")

class MedicalKnowledgeAgent(RetrievalAgent):
    """Retrieves from Vector and Semantic Databases (Adapted for Medicine)."""
    def __init__(self, vector_db: VectorDatabase, semantic_db: SemanticDatabase):
        super().__init__("MedicalKnowledgeAgent")
        self.vector_db = vector_db
        self.semantic_db = semantic_db

    def retrieve(self, query: str, query_type: str = "semantic") -> List[str]:
        results = []
        if query_type == "vector":
            # In a real system, 'query' would first be embedded, likely into a medical context embedding
            mock_embedding = [0.3] * 768 # Placeholder medical embedding
            results.extend(self.vector_db.query(mock_embedding))
        elif query_type == "semantic":
            results.extend(self.semantic_db.query(query))
        return results

class MedicalSearchAgent(RetrievalAgent):
    """Retrieves from external search engines (Adapted for Medicine)."""
    def __init__(self, search_engines: List[SearchEngine]):
        super().__init__("MedicalSearchAgent")
        self.search_engines = search_engines

    def retrieve(self, query: str) -> List[str]:
        all_results = []
        for engine in self.search_engines:
            all_results.extend(engine.search(query))
        return all_results


# --- 3. Agentic Reasoning and Console (Adapted for Medicine) ---

class AgenticMedicalReasoning:
    """Orchestrates LLM, LRM, and Retrieval Agents for Medical Tasks."""
    def __init__(self, llm_general: LargeLanguageModel,
                 lrm_medical: LargeReasoningModel, memory: Memory,
                 kb_agent: MedicalKnowledgeAgent, se_agent: MedicalSearchAgent):
        self.llm_general = llm_general
        self.lrm_medical = lrm_medical
        self.memory = memory
        self.kb_agent = kb_agent
        self.se_agent = se_agent
        # We'll use the general LLM as the primary interpreter
        self.current_llm = self.llm_general

    def process_query(self, modified_input: str) -> str:
        """
        This method embodies the core Agentic Reasoning loop for medical queries.
        """
        print(f"\n[Agentic Medical Reasoning] Processing input using {self.current_llm.identifier} and {self.lrm_medical.identifier}: '{modified_input[:80]}...'")
        self.memory.add_short_term(f"User query: {modified_input}")

        # Step 1: Initial LLM interpretation and plan using the general LLM
        llm_system_prompt_initial = "You are an AI medical assistant. Analyze the user query to determine information needs and initial analysis steps for medical tasks. Prioritize retrieving relevant medical data."
        initial_llm_thought = self.current_llm.generate(f"{llm_system_prompt_initial}\nUser query: {modified_input}")
        self.memory.add_short_term(f"LLM initial thought: {initial_llm_thought}")
        print(f"LLM Initial Thought: {initial_llm_thought}")

        # Step 2: Determine retrieval needs based on LLM's thought (Medical Keywords)
        retrieval_query = ""
        # Keywords updated for medicine
        medical_keywords = ["patient history", "symptoms", "lab results", "imaging", "medical record", "diagnosis", "treatment", "medication", "guidelines", "research", "clinical trial"]
        if any(keyword in initial_llm_thought.lower() for keyword in medical_keywords):
             retrieval_query = modified_input # Use original query or refine it
             print(f"Determined retrieval need for medical information based on: '{retrieval_query[:80]}'")

             # Perform Retrieval using Medical Agents
             kb_results = self.kb_agent.retrieve(retrieval_query, query_type="semantic") # Assume semantic for most medical queries
             se_results = self.se_agent.retrieve(retrieval_query)
             retrieved_info = "\n".join(kb_results + se_results)
             print(f"Retrieved Medical Information: {retrieved_info[:200]}...")
             self.memory.add_short_term(f"Retrieved: {retrieved_info}")
        else:
             retrieved_info = "No specific medical retrieval needed."
             print(retrieved_info)


        # Step 3: Engage LRM (Medical Reasoning Model) if complex reasoning is required
        lrm_system_prompt = "You are a specialized medical reasoning engine (e.g., Claude Sonnet). Analyze the gathered medical information and the user's original intent to formulate a precise medical analysis or recommendation (e.g., differential diagnoses, treatment plans, interpreting results). Focus on accuracy, evidence-based practices, and relevant medical context."
        # Keywords updated for medical reasoning tasks
        reasoning_keywords = ["analyze", "interpret", "evaluate", "diagnose", "suggest diagnoses", "recommend treatment", "compare", "predict", "manage condition"]
        if any(keyword in modified_input.lower() for keyword in reasoning_keywords) or any(keyword in initial_llm_thought.lower() for keyword in reasoning_keywords):
             reasoning_input = f"User query: {modified_input}\nInitial LLM thought: {initial_llm_thought}\nRetrieved info: {retrieved_info}"
             lrm_output = self.lrm_medical.generate(f"{lrm_system_prompt}\n{reasoning_input}")
             self.memory.add_short_term(f"LRM medical reasoning: {lrm_output}")
             print(f"LRM Medical Output: {lrm_output}")
        else:
             lrm_output = "No specific complex medical reasoning performed by the specialized model."
             print(lrm_output)


        # Step 4: Final LLM Synthesis (General LLM)
        final_llm_prompt = f"Based on the user's original medical query: '{modified_input}', initial analysis: '{initial_llm_thought}', retrieved medical information: '{retrieved_info}', and specialized medical reasoning by {self.lrm_medical.identifier}: '{lrm_output}', synthesize a comprehensive, accurate, and professional medical response using {self.current_llm.name}.  Remember to state this is not a substitute for professional medical advice."
        final_response = self.current_llm.generate(final_llm_prompt, max_tokens=1000)
        self.memory.add_long_term(f"Interaction: Query='{modified_input}', Response='{final_response}'")
        print(f"\n[Final Medical Response] {final_response}")
        return final_response

class AgenticMedicalConsole:
    """The top-level console orchestrating user interaction for Medical Queries."""
    def __init__(self, gemini_client: Optional[genai.GenerativeModel], claude_client: Optional[Anthropic]):
        # Initialize general LLM
        self.llm_general = LargeLanguageModel("Gemini 2.0 Flash (General)", 'gemini-2.0-flash', gemini_client)
        # Initialize Medical Reasoning Model
        self.lrm_medical = LargeReasoningModel("Claude Opus 4 (Financial Reasoning)", 'claude-opus-4-20250514', claude_client)

        self.memory = Memory() # Memory class remains general
        self.vector_db = VectorDatabase() # Mock DB adapted for medicine
        self.semantic_db = SemanticDatabase() # Mock DB adapted for medicine
        self.search_engines = [SearchEngine()] # Mock Search Engine adapted for medicine
        # Use Medical-specific Retrieval Agents
        self.kb_agent = MedicalKnowledgeAgent(self.vector_db, self.semantic_db)
        self.se_agent = MedicalSearchAgent(self.search_engines)

        # Use AgenticMedicalReasoning
        self.agentic_reasoning = AgenticMedicalReasoning(
            llm_general=self.llm_general,
            lrm_medical=self.lrm_medical,
            memory=self.memory,
            kb_agent=self.kb_agent,
            se_agent=self.se_agent
        )

    def run_query(self, user_query: str) -> str:
        print(f"\n--- [Agentic Medical Console] Processing User Query: '{user_query}' ---")
        parsed_query = user_query.strip()
        print(f"Query Parsed: '{parsed_query}'")

        # System Prompt for the overall medical task
        system_prompt = "You are a helpful AI medical assistant. Your goal is to provide accurate and comprehensive medical analysis and information based on the provided data and retrieved knowledge. Always state that this is not a substitute for professional medical advice."
        modified_input = f"{system_prompt}\nUser: {parsed_query}"

        # Agentic Reasoning for Medical Query
        compiled_output = self.agentic_reasoning.process_query(modified_input)

        print(f"--- [Agentic Medical Console] Query Complete ---")
        return compiled_output

# --- Example Usage (Adapted for Medical Query) ---
if __name__ == "__main__":
    print("Setting up API clients...")
    # --- Gemini API Setup ---
    GOOGLE_API_KEY = None
    if IN_COLAB:
        try:
            GOOGLE_API_KEY = userdata.get('GEMINI')
            print("Successfully retrieved GEMINI_API_KEY from Colab environment variable.")
        except Exception as e:
            print(f"Could not retrieve GEMINI_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not GOOGLE_API_KEY:
        GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY") # Fallback to environment variable
        if GOOGLE_API_KEY:
            print("Successfully retrieved GOOGLE_API_KEY from environment variable.")
        else:
            print("WARNING: GOOGLE_API_KEY not found. Gemini API calls will be mocked.")

    gemini_client = None
    if GOOGLE_API_KEY:
        try:
            genai.configure(api_key=GOOGLE_API_KEY)
            gemini_client = genai.GenerativeModel('gemini-2.0-flash')
            print("Gemini GenerativeModel ('gemini-2.0-flash') client initialized.")
        except Exception as e:
            print(f"ERROR initializing Gemini client: {e}. Gemini API calls will be mocked.")
    else:
        print("No Google API Key found, Gemini client will not be initialized.")

    # --- Claude API Setup ---
    CLAUDE_API_KEY = None
    if IN_COLAB:
        try:
            CLAUDE_API_KEY = os.environ.get("CLAUDE3_API_KEY")
            print("Successfully retrieved CLAUDE3_API_KEY from Colab userdata.")
        except Exception as e:
            print(f"Could not retrieve CLAUDE3_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not CLAUDE_API_KEY:
        CLAUDE_API_KEY = os.environ.get("ANTHROPIC_API_KEY") # Fallback to environment variable
        if CLAUDE_API_KEY:
            print("Successfully retrieved ANTHROPIC_API_KEY from environment variable.")
        else:
            print("WARNING: ANTHROPIC_API_KEY not found. Claude API calls will be mocked.")

    claude_client = None
    if CLAUDE_API_KEY:
        try:
            claude_client = Anthropic(api_key=CLAUDE_API_KEY)
            print("Anthropic Claude client initialized.")
        except Exception as e:
            print(f"ERROR initializing Anthropic client: {e}. Claude API calls will be mocked.")
    else:
        print("No Anthropic API Key found, Claude client will not be initialized.")


    # Instantiate the console with the prepared API clients
    # This line creates the main console object that orchestrates everything.
    console = AgenticMedicalConsole(gemini_client=gemini_client, claude_client=claude_client)
    print('\n')
    print("\nAgenticMedicalConsole initialized. Running your specific medical query...")
    print('\n')

    # --- Running Your Specific Medical Query ---
    your_medical_query = "Analyze the patient's symptoms and lab results to suggest potential diagnoses and next steps."

    print(f"\n--- Running Medical Query: {your_medical_query} ---")
    medical_output = console.run_query(your_medical_query)
    print(f"\n--- Medical Analysis Output ---")
    print(medical_output)
    print("--- End of Medical Analysis Output ---")

    # --- Displaying Memory Contents after your query ---
    print("\n--- Current Short-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_short_term())
    print('\n')

    print("\n--- Current Long-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_long_term())
    print('\n')

Setting up API clients...
Successfully retrieved GEMINI_API_KEY from Colab environment variable.
Gemini GenerativeModel ('gemini-2.0-flash') client initialized.
Successfully retrieved CLAUDE3_API_KEY from Colab userdata.
Anthropic Claude client initialized.



AgenticMedicalConsole initialized. Running your specific medical query...



--- Running Medical Query: Analyze the patient's symptoms and lab results to suggest potential diagnoses and next steps. ---

--- [Agentic Medical Console] Processing User Query: 'Analyze the patient's symptoms and lab results to suggest potential diagnoses and next steps.' ---
Query Parsed: 'Analyze the patient's symptoms and lab results to suggest potential diagnoses and next steps.'

[Agentic Medical Reasoning] Processing input using gemini-2.0-flash and claude-opus-4-20250514: 'You are a helpful AI medical assistant. Your goal is to provide accurate and com...'
[Gemini 2.0 Flash (General) (gemini-2.0-flash)] Attempting to generate response for prompt